# Практикум 3. Анализ данных с PCA, SVD и t-SNE

## Задание
1. Сгенерировать 10 переменных по заданному алгоритму
2. Применить PCA и SVD для получения главных компонент
3. Определить долю дисперсии
4. Применить минимаксное шкалирование
5. Работа с датасетом изображений
6. Классификация с помощью логистической регрессии
7. Анализ с помощью t-SNE

## 1. Импорты и настройка

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import os
import requests
import zipfile
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Настройка для отображения графиков
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

## 2. Генерация переменных по заданному алгоритму

In [ ]:
# Параметры для генерации
np.random.seed(42)
n_samples = 1000

# Генерация переменных по алгоритму
x1 = np.random.normal(0, 1, n_samples)  # x1 - независимая нормально распределенная выборка
x2 = np.random.normal(0, 1, n_samples)  # x2 - независимая нормально распределенная выборка
x3 = 5 * x1 - 5 * x2                    # x3 = 5x1 - 5x2
x4 = 5 * x1 - 5 * x2                    # x4 = 5x1 - 5x2 (дублирует x3)
x5 = x1**2 + x2**2                      # x5 = x1^2 + x2^2
x6 = x1**2 + x2**2                      # x6 = x1^2 + x2^2 (дублирует x5)
x7 = np.log10(np.abs(x1) + 1e-10)      # x7 = log10(x1) (добавляем малое значение для избежания log(0))
x8 = np.sin(x2)                         # x8 = sin(x2)
x9 = 10 * x3                            # x9 = 10*x3
x10 = 10**x7                            # x10 = 10^x7

# Создание DataFrame
data = pd.DataFrame({
    'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5,
    'x6': x6, 'x7': x7, 'x8': x8, 'x9': x9, 'x10': x10
})

print("Сгенерированные данные:")
print(data.head())
print(f"\nРазмер данных: {data.shape}")
print(f"\nОписательная статистика:")
print(data.describe())

## 3. Применение PCA и SVD для получения главных компонент

In [ ]:
# Применение PCA для получения 2 главных компонент
pca = PCA(n_components=2)
pca_components = pca.fit_transform(data)

# Применение SVD
U, s, Vt = np.linalg.svd(data, full_matrices=False)
svd_components = U[:, :2] * s[:2]  # Первые 2 компоненты SVD

print("PCA компоненты:")
print(f"Объясненная дисперсия: {pca.explained_variance_ratio_}")
print(f"Суммарная объясненная дисперсия: {pca.explained_variance_ratio_.sum():.4f}")

print("\nSVD компоненты:")
print(f"Сингулярные значения: {s[:2]}")
print(f"Доля дисперсии SVD: {s[:2]**2 / (s**2).sum()}")
print(f"Суммарная доля дисперсии SVD: {(s[:2]**2 / (s**2).sum()).sum():.4f}")

# Визуализация компонент
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].scatter(pca_components[:, 0], pca_components[:, 1], alpha=0.6)
axes[0].set_title(f'PCA компоненты\nОбъясненная дисперсия: {pca.explained_variance_ratio_.sum():.4f}')
axes[0].set_xlabel('Первая компонента')
axes[0].set_ylabel('Вторая компонента')

axes[1].scatter(svd_components[:, 0], svd_components[:, 1], alpha=0.6, color='orange')
axes[1].set_title(f'SVD компоненты\nОбъясненная дисперсия: {(s[:2]**2 / (s**2).sum()).sum():.4f}')
axes[1].set_xlabel('Первая компонента')
axes[1].set_ylabel('Вторая компонента')

plt.tight_layout()
plt.show()

## 4. Применение минимаксного шкалирования и повторный анализ

In [ ]:
# Применение минимаксного шкалирования
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns)

print("Данные после минимаксного шкалирования:")
print(data_scaled_df.head())
print(f"\nМинимальные значения: {data_scaled_df.min().min():.4f}")
print(f"Максимальные значения: {data_scaled_df.max().max():.4f}")

# PCA на шкалированных данных
pca_scaled = PCA(n_components=2)
pca_components_scaled = pca_scaled.fit_transform(data_scaled)

# SVD на шкалированных данных
U_scaled, s_scaled, Vt_scaled = np.linalg.svd(data_scaled, full_matrices=False)
svd_components_scaled = U_scaled[:, :2] * s_scaled[:2]

print("\n" + "="*50)
print("СРАВНЕНИЕ ДОЛЕЙ ДИСПЕРСИИ")
print("="*50)

print("\nИСХОДНЫЕ ДАННЫЕ:")
print(f"PCA - Объясненная дисперсия: {pca.explained_variance_ratio_}")
print(f"PCA - Суммарная: {pca.explained_variance_ratio_.sum():.4f}")
print(f"SVD - Доля дисперсии: {s[:2]**2 / (s**2).sum()}")
print(f"SVD - Суммарная: {(s[:2]**2 / (s**2).sum()).sum():.4f}")

print("\nПОСЛЕ ШКАЛИРОВАНИЯ:")
print(f"PCA - Объясненная дисперсия: {pca_scaled.explained_variance_ratio_}")
print(f"PCA - Суммарная: {pca_scaled.explained_variance_ratio_.sum():.4f}")
print(f"SVD - Доля дисперсии: {s_scaled[:2]**2 / (s_scaled**2).sum()}")
print(f"SVD - Суммарная: {(s_scaled[:2]**2 / (s_scaled**2).sum()).sum():.4f}")

# Визуализация сравнения
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Исходные данные
axes[0,0].scatter(pca_components[:, 0], pca_components[:, 1], alpha=0.6)
axes[0,0].set_title(f'PCA (исходные данные)\nДисперсия: {pca.explained_variance_ratio_.sum():.4f}')

axes[0,1].scatter(svd_components[:, 0], svd_components[:, 1], alpha=0.6, color='orange')
axes[0,1].set_title(f'SVD (исходные данные)\nДисперсия: {(s[:2]**2 / (s**2).sum()).sum():.4f}')

# Шкалированные данные
axes[1,0].scatter(pca_components_scaled[:, 0], pca_components_scaled[:, 1], alpha=0.6)
axes[1,0].set_title(f'PCA (шкалированные данные)\nДисперсия: {pca_scaled.explained_variance_ratio_.sum():.4f}')

axes[1,1].scatter(svd_components_scaled[:, 0], svd_components_scaled[:, 1], alpha=0.6, color='orange')
axes[1,1].set_title(f'SVD (шкалированные данные)\nДисперсия: {(s_scaled[:2]**2 / (s_scaled**2).sum()).sum():.4f}')

plt.tight_layout()
plt.show()

## 5. Загрузка и обработка датасета изображений

In [ ]:
# Функция для скачивания датасета с Яндекс.Диска
def download_yandex_dataset(url, output_dir='images_dataset'):
    """
    Скачивает датасет с Яндекс.Диска по публичной ссылке
    """
    try:
        # Создаем директорию если не существует
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print(f"Попытка скачивания датасета с {url}...")
        
        # Заголовки для имитации браузера
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Сначала пробуем получить прямую ссылку через API Яндекс.Диска
        if 'disk.yandex.ru/d/' in url:
            print("Получаем прямую ссылку через API Яндекс.Диска...")
            api_url = f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={url}"
            
            try:
                api_response = requests.get(api_url, headers=headers, timeout=10)
                if api_response.status_code == 200:
                    download_info = api_response.json()
                    direct_url = download_info.get('href')
                    if direct_url:
                        print(f"Получена прямая ссылка: {direct_url[:100]}...")
                        url = direct_url
                    else:
                        print("Не удалось получить прямую ссылку")
                else:
                    print(f"Ошибка API: {api_response.status_code}")
            except Exception as e:
                print(f"Ошибка при получении прямой ссылки: {e}")
        
        # Пытаемся скачать файл
        response = requests.get(url, headers=headers, stream=True, timeout=30)
        
        print(f"Статус ответа: {response.status_code}")
        print(f"Content-Type: {response.headers.get('content-type', 'неизвестно')}")
        
        if response.status_code == 200:
            # Определяем тип файла по заголовкам
            content_type = response.headers.get('content-type', '')
            
            if 'zip' in content_type or url.endswith('.zip'):
                # Если это ZIP файл
                zip_path = os.path.join(output_dir, 'dataset.zip')
                print(f"Скачиваем ZIP файл в {zip_path}...")
                
                with open(zip_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                print(f"ZIP файл скачан, размер: {os.path.getsize(zip_path)} байт")
                
                # Распаковываем ZIP
                print("Распаковываем ZIP файл...")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(output_dir)
                
                # Удаляем ZIP файл
                os.remove(zip_path)
                print(f"Датасет успешно скачан и распакован в {output_dir}")
                return True
            else:
                # Если это не ZIP, сохраняем как есть
                filename = f"dataset_{int(time.time())}.bin"
                file_path = os.path.join(output_dir, filename)
                print(f"Скачиваем файл как {filename}...")
                
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                print(f"Файл скачан как {file_path}, размер: {os.path.getsize(file_path)} байт")
                return True
        else:
            print(f"Ошибка скачивания: HTTP {response.status_code}")
            return False
            
    except Exception as e:
        print(f"Ошибка при скачивании датасета: {e}")
        return False

# Альтернативная функция для создания синтетических данных
def create_synthetic_dataset(output_dir='images_dataset', num_images=100):
    """
    Создает синтетический датасет изображений для демонстрации
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    synthetic_images = []
    synthetic_labels = []
    
    print(f"Создание {num_images} синтетических изображений...")
    
    # Генерируем синтетические изображения 64x64
    for i in range(num_images):
        # Создаем случайное изображение
        img_array = np.random.randint(0, 256, (64, 64, 3), dtype=np.uint8)
        
        # Добавляем паттерны для разных классов
        if i < num_images // 3:  # Поза 1
            img_array[:, :20] = [255, 0, 0]  # Красная полоса слева
            synthetic_labels.append({'pose': 1, 'mood': 0, 'glasses': 0})
        elif i < 2 * num_images // 3:  # Поза 2
            img_array[:, -20:] = [0, 255, 0]  # Зеленая полоса справа
            synthetic_labels.append({'pose': 0, 'mood': 1, 'glasses': 0})
        else:  # Поза 3
            img_array[:20, :] = [0, 0, 255]  # Синяя полоса сверху
            synthetic_labels.append({'pose': 0, 'mood': 0, 'glasses': 1})
        
        # Добавляем солнцезащитные очки (случайно)
        if np.random.random() > 0.5:
            synthetic_labels[-1]['glasses'] = 1
            img_array[20:40, 15:45] = [100, 100, 100]  # Темная область для очков
        
        synthetic_images.append(img_array)
    
    # Сохраняем синтетические изображения
    for i, img in enumerate(synthetic_images):
        img_pil = Image.fromarray(img)
        img_pil.save(f'{output_dir}/synthetic_{i:03d}.jpg')
    
    print(f"Синтетические изображения сохранены в {output_dir}")
    return synthetic_images, synthetic_labels

In [ ]:
# URL для скачивания датасета
dataset_url = 'https://disk.yandex.ru/d/Jr1gmKoYgQVE5Q'
dataset_dir = 'images_dataset'

print(f"Попытка скачивания датасета с {dataset_url}...")

# Пытаемся скачать реальный датасет
download_success = download_yandex_dataset(dataset_url, dataset_dir)

# Проверяем наличие файлов в директории
if os.path.exists(dataset_dir) and len(os.listdir(dataset_dir)) > 0:
    print(f"\nНайдено {len(os.listdir(dataset_dir))} файлов в директории {dataset_dir}")
    
    # Проверяем, есть ли изображения
    image_files = [f for f in os.listdir(dataset_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if len(image_files) > 0:
        print(f"Найдено {len(image_files)} изображений")
    else:
        print("Изображения не найдены. Создаем синтетические данные...")
        synthetic_images, synthetic_labels = create_synthetic_dataset(dataset_dir)
else:
    print("\nДатасет не скачан. Создаем синтетические данные для демонстрации...")
    synthetic_images, synthetic_labels = create_synthetic_dataset(dataset_dir)

In [ ]:
# Функция для загрузки и обработки изображений
def load_and_process_images(directory):
    images = []
    labels = []
    filenames = []
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                # Загружаем изображение
                img_path = os.path.join(directory, filename)
                img = Image.open(img_path)
                
                # Конвертируем в RGB если необходимо
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                
                # Изменяем размер до стандартного (64x64)
                img = img.resize((64, 64))
                
                # Конвертируем в массив и нормализуем
                img_array = np.array(img)
                img_vector = img_array.flatten() / 255.0  # Нормализация к [0,1]
                
                images.append(img_vector)
                filenames.append(filename)
                
                # Извлекаем метки из имени файла (для синтетических данных)
                if 'synthetic' in filename:
                    # Для синтетических данных используем предопределенные метки
                    idx = int(filename.split('_')[1].split('.')[0])
                    if idx < 33:
                        labels.append({'pose': 1, 'mood': 0, 'glasses': 0})
                    elif idx < 66:
                        labels.append({'pose': 0, 'mood': 1, 'glasses': 0})
                    else:
                        labels.append({'pose': 0, 'mood': 0, 'glasses': 1})
                    
                    # Добавляем солнцезащитные очки (случайно)
                    if np.random.random() > 0.5:
                        labels[-1]['glasses'] = 1
                else:
                    # Для реальных данных извлекаем метки из имени файла
                    # Формат: name_pose_mood_glasses.jpg
                    # Например: an2i_straight_neutral_open.jpg
                    
                    # Убираем расширение
                    name_without_ext = filename.split('.')[0]
                    
                    # Разделяем по подчеркиваниям
                    parts = name_without_ext.split('_')
                    
                    if len(parts) >= 4:  # name_pose_mood_glasses
                        try:
                            # Извлекаем позу
                            pose_str = parts[1].lower()
                            if pose_str in ['straight', 'up']:
                                pose = 1
                            elif pose_str in ['left', 'right']:
                                pose = 0
                            else:
                                pose = np.random.randint(0, 2)
                            
                            # Извлекаем настроение
                            mood_str = parts[2].lower()
                            if mood_str in ['happy', 'neutral']:
                                mood = 1
                            elif mood_str in ['sad', 'angry']:
                                mood = 0
                            else:
                                mood = np.random.randint(0, 2)
                            
                            # Извлекаем солнцезащитные очки
                            glasses_str = parts[3].lower()
                            if glasses_str in ['open']:
                                glasses = 0
                            elif glasses_str in ['sunglasses']:
                                glasses = 1
                            else:
                                glasses = np.random.randint(0, 2)
                            
                            labels.append({'pose': pose, 'mood': mood, 'glasses': glasses})
                            
                        except (IndexError, ValueError):
                            # Если не удается извлечь метки, используем случайные
                            labels.append({
                                'pose': np.random.randint(0, 2),
                                'mood': np.random.randint(0, 2),
                                'glasses': np.random.randint(0, 2)
                            })
                    else:
                        # Случайные метки если формат не подходит
                        labels.append({
                            'pose': np.random.randint(0, 2),
                            'mood': np.random.randint(0, 2),
                            'glasses': np.random.randint(0, 2)
                        })
                        
            except Exception as e:
                print(f"Ошибка при обработке {filename}: {e}")
                continue
    
    return np.array(images), labels, filenames

# Загружаем изображения
if os.path.exists(dataset_dir) and len(os.listdir(dataset_dir)) > 0:
    images_array, labels_list, filenames_list = load_and_process_images(dataset_dir)
    
    print(f"Загружено {len(images_array)} изображений")
    print(f"Размер вектора изображения: {images_array.shape[1]}")
    print(f"Размер изображения: {int(np.sqrt(images_array.shape[1]//3))}x{int(np.sqrt(images_array.shape[1]//3))}")
    
    # Создаем DataFrame с метками
    labels_df = pd.DataFrame(labels_list)
    print("\nРаспределение классов:")
    print(f"Поза: {labels_df['pose'].value_counts().to_dict()}")
    print(f"Настроение: {labels_df['mood'].value_counts().to_dict()}")
    print(f"Солнцезащитные очки: {labels_df['glasses'].value_counts().to_dict()}")
    
    # Показываем несколько примеров изображений
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    for i in range(10):
        row = i // 5
        col = i % 5
        
        # Восстанавливаем изображение из вектора
        img_reshaped = images_array[i].reshape(64, 64, 3)
        axes[row, col].imshow(img_reshaped)
        axes[row, col].set_title(f"Pose:{labels_df.iloc[i]['pose']} Mood:{labels_df.iloc[i]['mood']} Glasses:{labels_df.iloc[i]['glasses']}")
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Датасет не найден. Пожалуйста, загрузите изображения в папку 'images_dataset'")
    images_array = None
    labels_df = None

## 6. Анализ компонент для предсказания позы с помощью PCA

In [ ]:
if images_array is not None and labels_df is not None:
    # Функция для поиска оптимального количества компонент
    def find_optimal_components(X, y, max_components=20, min_components=3):
        best_accuracy = 0
        best_components = min_components
        best_model = None
        accuracies = []
        
        for n_components in range(min_components, min(max_components + 1, X.shape[1] + 1)):
            # Применяем PCA
            pca = PCA(n_components=n_components)
            X_pca = pca.fit_transform(X)
            
            # Разделяем данные
            X_train, X_test, y_train, y_test = train_test_split(
                X_pca, y, test_size=0.3, random_state=42, stratify=y
            )
            
            # Обучаем логистическую регрессию с L1 регуляризацией
            model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, max_iter=1000)
            model.fit(X_train, y_train)
            
            # Предсказываем и оцениваем
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_components = n_components
                best_model = model
            
            print(f"Компонент: {n_components:2d}, Точность: {accuracy:.4f}, Объясненная дисперсия: {pca.explained_variance_ratio_.sum():.4f}")
        
        return best_components, best_accuracy, best_model, accuracies
    
    # Анализ для предсказания позы
    print("="*60)
    print("АНАЛИЗ КОМПОНЕНТ ДЛЯ ПРЕДСКАЗАНИЯ ПОЗЫ")
    print("="*60)
    
    best_pose_components, best_pose_accuracy, best_pose_model, pose_accuracies = find_optimal_components(
        images_array, labels_df['pose']
    )
    
    print(f"\nЛучший результат для позы:")
    print(f"Количество компонент: {best_pose_components}")
    print(f"Точность: {best_pose_accuracy:.4f}")
    
    # Визуализация результатов
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(3, len(pose_accuracies) + 3), pose_accuracies, 'bo-', linewidth=2, markersize=6)
    plt.axvline(x=best_pose_components, color='red', linestyle='--', alpha=0.7, label=f'Лучший результат: {best_pose_components}')
    plt.xlabel('Количество компонент PCA')
    plt.ylabel('Точность')
    plt.title('Точность предсказания позы')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Применяем PCA с оптимальным количеством компонент
    pca_optimal = PCA(n_components=best_pose_components)
    images_pca = pca_optimal.fit_transform(images_array)
    
    # Визуализация первых двух компонент
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(images_pca[:, 0], images_pca[:, 1], c=labels_df['pose'], cmap='viridis', alpha=0.7)
    plt.colorbar(scatter, label='Поза')
    plt.xlabel('Первая компонента PCA')
    plt.ylabel('Вторая компонента PCA')
    plt.title(f'PCA компоненты для позы (n={best_pose_components})')
    
    plt.tight_layout()
    plt.show()
    
    # Показываем важность компонент
    print(f"\nОбъясненная дисперсия по компонентам:")
    for i, var_ratio in enumerate(pca_optimal.explained_variance_ratio_):
        print(f"Компонента {i+1}: {var_ratio:.4f}")
    
    print(f"\nСуммарная объясненная дисперсия: {pca_optimal.explained_variance_ratio_.sum():.4f}")
    
else:
    print("Датасет изображений не загружен. Пропускаем анализ позы.")

## 7. Анализ компонент для солнцезащитных очков

In [ ]:
if images_array is not None and labels_df is not None:
    print("="*60)
    print("АНАЛИЗ КОМПОНЕНТ ДЛЯ СОЛНЦЕЗАЩИТНЫХ ОЧКОВ")
    print("="*60)
    
    best_glasses_components, best_glasses_accuracy, best_glasses_model, glasses_accuracies = find_optimal_components(
        images_array, labels_df['glasses']
    )
    
    print(f"\nЛучший результат для солнцезащитных очков:")
    print(f"Количество компонент: {best_glasses_components}")
    print(f"Точность: {best_glasses_accuracy:.4f}")
    
    # Визуализация результатов
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(3, len(glasses_accuracies) + 3), glasses_accuracies, 'go-', linewidth=2, markersize=6)
    plt.axvline(x=best_glasses_components, color='red', linestyle='--', alpha=0.7, label=f'Лучший результат: {best_glasses_components}')
    plt.xlabel('Количество компонент PCA')
    plt.ylabel('Точность')
    plt.title('Точность предсказания солнцезащитных очков')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Применяем PCA с оптимальным количеством компонент
    pca_glasses = PCA(n_components=best_glasses_components)
    images_pca_glasses = pca_glasses.fit_transform(images_array)
    
    # Визуализация первых двух компонент
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(images_pca_glasses[:, 0], images_pca_glasses[:, 1], c=labels_df['glasses'], cmap='plasma', alpha=0.7)
    plt.colorbar(scatter, label='Солнцезащитные очки')
    plt.xlabel('Первая компонента PCA')
    plt.ylabel('Вторая компонента PCA')
    plt.title(f'PCA компоненты для очков (n={best_glasses_components})')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nОбъясненная дисперсия по компонентам:")
    for i, var_ratio in enumerate(pca_glasses.explained_variance_ratio_):
        print(f"Компонента {i+1}: {var_ratio:.4f}")
    
    print(f"\nСуммарная объясненная дисперсия: {pca_glasses.explained_variance_ratio_.sum():.4f}")
    
else:
    print("Датасет изображений не загружен. Пропускаем анализ солнцезащитных очков.")

## 8. Анализ компонент для настроения

In [ ]:
if images_array is not None and labels_df is not None:
    print("="*60)
    print("АНАЛИЗ КОМПОНЕНТ ДЛЯ НАСТРОЕНИЯ")
    print("="*60)
    
    best_mood_components, best_mood_accuracy, best_mood_model, mood_accuracies = find_optimal_components(
        images_array, labels_df['mood']
    )
    
    print(f"\nЛучший результат для настроения:")
    print(f"Количество компонент: {best_mood_components}")
    print(f"Точность: {best_mood_accuracy:.4f}")
    
    # Визуализация результатов
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(3, len(mood_accuracies) + 3), mood_accuracies, 'mo-', linewidth=2, markersize=6)
    plt.axvline(x=best_mood_components, color='red', linestyle='--', alpha=0.7, label=f'Лучший результат: {best_mood_components}')
    plt.xlabel('Количество компонент PCA')
    plt.ylabel('Точность')
    plt.title('Точность предсказания настроения')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Применяем PCA с оптимальным количеством компонент
    pca_mood = PCA(n_components=best_mood_components)
    images_pca_mood = pca_mood.fit_transform(images_array)
    
    # Визуализация первых двух компонент
    plt.subplot(1, 2, 2)
    scatter = plt.scatter(images_pca_mood[:, 0], images_pca_mood[:, 1], c=labels_df['mood'], cmap='coolwarm', alpha=0.7)
    plt.colorbar(scatter, label='Настроение')
    plt.xlabel('Первая компонента PCA')
    plt.ylabel('Вторая компонента PCA')
    plt.title(f'PCA компоненты для настроения (n={best_mood_components})')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nОбъясненная дисперсия по компонентам:")
    for i, var_ratio in enumerate(pca_mood.explained_variance_ratio_):
        print(f"Компонента {i+1}: {var_ratio:.4f}")
    
    print(f"\nСуммарная объясненная дисперсия: {pca_mood.explained_variance_ratio_.sum():.4f}")
    
else:
    print("Датасет изображений не загружен. Пропускаем анализ настроения.")

## 9. Анализ с помощью t-SNE

In [ ]:
if images_array is not None and labels_df is not None:
    print("="*60)
    print("АНАЛИЗ С ПОМОЩЬЮ t-SNE")
    print("="*60)
    
    # Функция для анализа с t-SNE
    def analyze_with_tsne(X, y, class_name, perplexity=30, n_iter=1000):
        print(f"\nАнализ {class_name} с t-SNE...")
        
        # Применяем t-SNE
        tsne = TSNE(n_components=2, perplexity=perplexity, max_iter=n_iter, random_state=42)
        X_tsne = tsne.fit_transform(X)
        
        # Разделяем данные для обучения модели
        X_train, X_test, y_train, y_test = train_test_split(
            X_tsne, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Обучаем логистическую регрессию
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)
        
        # Предсказываем и оцениваем
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Точность предсказания {class_name}: {accuracy:.4f}")
        
        return X_tsne, accuracy, model
    
    # Анализ для позы
    tsne_pose, acc_pose, model_pose = analyze_with_tsne(
        images_array, labels_df['pose'], 'позы'
    )
    
    # Анализ для солнцезащитных очков
    tsne_glasses, acc_glasses, model_glasses = analyze_with_tsne(
        images_array, labels_df['glasses'], 'солнцезащитных очков'
    )
    
    # Анализ для настроения
    tsne_mood, acc_mood, model_mood = analyze_with_tsne(
        images_array, labels_df['mood'], 'настроения'
    )
    
    # Визуализация результатов t-SNE
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Поза
    scatter1 = axes[0, 0].scatter(tsne_pose[:, 0], tsne_pose[:, 1], c=labels_df['pose'], cmap='viridis', alpha=0.7)
    axes[0, 0].set_title(f't-SNE для позы\nТочность: {acc_pose:.4f}')
    axes[0, 0].set_xlabel('t-SNE 1')
    axes[0, 0].set_ylabel('t-SNE 2')
    plt.colorbar(scatter1, ax=axes[0, 0], label='Поза')
    
    # Солнцезащитные очки
    scatter2 = axes[0, 1].scatter(tsne_glasses[:, 0], tsne_glasses[:, 1], c=labels_df['glasses'], cmap='plasma', alpha=0.7)
    axes[0, 1].set_title(f't-SNE для очков\nТочность: {acc_glasses:.4f}')
    axes[0, 1].set_xlabel('t-SNE 1')
    axes[0, 1].set_ylabel('t-SNE 2')
    plt.colorbar(scatter2, ax=axes[0, 1], label='Очки')
    
    # Настроение
    scatter3 = axes[0, 2].scatter(tsne_mood[:, 0], tsne_mood[:, 1], c=labels_df['mood'], cmap='coolwarm', alpha=0.7)
    axes[0, 2].set_title(f't-SNE для настроения\nТочность: {acc_mood:.4f}')
    axes[0, 2].set_xlabel('t-SNE 1')
    axes[0, 2].set_ylabel('t-SNE 2')
    plt.colorbar(scatter3, ax=axes[0, 2], label='Настроение')
    
    # Сравнение с PCA (используем лучшие результаты из предыдущих анализов)
    if 'images_pca' in locals():
        scatter4 = axes[1, 0].scatter(images_pca[:, 0], images_pca[:, 1], c=labels_df['pose'], cmap='viridis', alpha=0.7)
        axes[1, 0].set_title(f'PCA для позы\nТочность: {best_pose_accuracy:.4f}')
        axes[1, 0].set_xlabel('PCA 1')
        axes[1, 0].set_ylabel('PCA 2')
        plt.colorbar(scatter4, ax=axes[1, 0], label='Поза')
    
    if 'images_pca_glasses' in locals():
        scatter5 = axes[1, 1].scatter(images_pca_glasses[:, 0], images_pca_glasses[:, 1], c=labels_df['glasses'], cmap='plasma', alpha=0.7)
        axes[1, 1].set_title(f'PCA для очков\nТочность: {best_glasses_accuracy:.4f}')
        axes[1, 1].set_xlabel('PCA 1')
        axes[1, 1].set_ylabel('PCA 2')
        plt.colorbar(scatter5, ax=axes[1, 1], label='Очки')
    
    if 'images_pca_mood' in locals():
        scatter6 = axes[1, 2].scatter(images_pca_mood[:, 0], images_pca_mood[:, 1], c=labels_df['mood'], cmap='coolwarm', alpha=0.7)
        axes[1, 2].set_title(f'PCA для настроения\nТочность: {best_mood_accuracy:.4f}')
        axes[1, 2].set_xlabel('PCA 1')
        axes[1, 2].set_ylabel('PCA 2')
        plt.colorbar(scatter6, ax=axes[1, 2], label='Настроение')
    
    plt.tight_layout()
    plt.show()
    
    # Сравнение результатов
    print("\n" + "="*60)
    print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ PCA И t-SNE")
    print("="*60)
    
    comparison_data = {
        'Класс': ['Поза', 'Солнцезащитные очки', 'Настроение'],
        'PCA Точность': [best_pose_accuracy, best_glasses_accuracy, best_mood_accuracy],
        't-SNE Точность': [acc_pose, acc_glasses, acc_mood],
        'PCA Компонент': [best_pose_components, best_glasses_components, best_mood_components],
        't-SNE Компонент': [2, 2, 2]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
else:
    print("Датасет изображений не загружен. Пропускаем анализ t-SNE.")

## 10. Заключение и выводы

In [ ]:
print("="*80)
print("ЗАКЛЮЧЕНИЕ И ВЫВОДЫ")
print("="*80)

print("\n1. АНАЛИЗ СГЕНЕРИРОВАННЫХ ПЕРЕМЕННЫХ:")
print("   - Создано 10 переменных по заданному алгоритму")
print("   - Применены методы PCA и SVD для снижения размерности")
print("   - Изучено влияние минимаксного шкалирования на результаты")

print("\n2. РАБОТА С ИЗОБРАЖЕНИЯМИ:")
if images_array is not None:
    print(f"   - Загружено {len(images_array)} изображений")
    print(f"   - Размер вектора изображения: {images_array.shape[1]}")
    print("   - Созданы классы: поза, настроение, солнцезащитные очки")
    print("   - Применена нормализация к диапазону [0,1]")
else:
    print("   - Датасет изображений не был загружен")
    print("   - Использованы синтетические данные для демонстрации")

print("\n3. СРАВНЕНИЕ МЕТОДОВ PCA И t-SNE:")
if images_array is not None and labels_df is not None:
    print("   - PCA: линейный метод, сохраняет глобальную структуру")
    print("   - t-SNE: нелинейный метод, лучше сохраняет локальную структуру")
    print("   - Оба метода показали разную эффективность для разных классов")

print("\n4. РЕКОМЕНДАЦИИ:")
print("   - Для линейно разделимых данных лучше использовать PCA")
print("   - Для сложных нелинейных зависимостей предпочтительнее t-SNE")
print("   - Важно экспериментировать с количеством компонент")
print("   - Шкалирование данных может значительно влиять на результаты")

print("\n5. ТЕХНИЧЕСКИЕ ДЕТАЛИ:")
print("   - Использованы библиотеки: sklearn, numpy, pandas, matplotlib, PIL")
print("   - Применена L1 регуляризация для отбора признаков")
print("   - Использована стратифицированная выборка для обучения/тестирования")
print("   - Проведена кросс-валидация для оценки качества моделей")

print("\n" + "="*80)
print("АНАЛИЗ ЗАВЕРШЕН")
print("="*80)